```{contents}
```
## **LLM Node in LangGraph**

An **LLM Node** in LangGraph is a specialized **graph node** whose primary responsibility is to perform **language model inference** and write the result into the shared **graph state**.
It is the fundamental building block for reasoning, planning, dialogue, and decision-making in LangGraph workflows.

---

### **1. Conceptual Role of an LLM Node**

An LLM node transforms **state → text → new state**.

```
(State) → Prompt Construction → LLM Inference → Parsing → State Update
```

It behaves like a **pure function over state**, except the computation is delegated to a language model.

---

### **2. Why LLM Nodes Are First-Class in LangGraph**

LLM nodes allow the graph to:

| Capability | Description               |
| ---------- | ------------------------- |
| Reason     | Interpret complex inputs  |
| Plan       | Decompose tasks           |
| Decide     | Choose next actions       |
| Generate   | Produce text/code         |
| Reflect    | Self-critique and improve |

Without LLM nodes, LangGraph would only orchestrate deterministic programs.

---

### **3. Anatomy of an LLM Node**

An LLM node consists of:

| Component    | Purpose                           |
| ------------ | --------------------------------- |
| Prompt       | Instructions + context            |
| Model        | LLM backend                       |
| Parser       | Converts text → structured output |
| State Mapper | Writes parsed output into state   |

---

### **4. Basic LLM Node Example**

```python
from langgraph.graph import StateGraph
from typing import TypedDict
from langchain_openai import ChatOpenAI

class State(TypedDict):
    question: str
    answer: str

llm = ChatOpenAI(model="gpt-4o-mini")

def llm_node(state: State):
    prompt = f"Answer this question: {state['question']}"
    response = llm.invoke(prompt)
    return {"answer": response.content}

builder = StateGraph(State)
builder.add_node("llm", llm_node)
```

The node:

1. Reads `state["question"]`
2. Calls the LLM
3. Writes to `state["answer"]`

---

### **5. Structured Output with LLM Nodes**

For production systems, outputs should be **structured**.

```python
from pydantic import BaseModel

class Output(BaseModel):
    decision: str
    confidence: float

def llm_node(state):
    response = llm.with_structured_output(Output).invoke(state["input"])
    return response.dict()
```

---

### **6. LLM Nodes with Tools**

LLM nodes may **invoke tools**:

```python
from langchain.tools import tool

@tool
def search(query: str) -> str:
    ...

llm = ChatOpenAI().bind_tools([search])
```

The node becomes a **controller** that decides when and how tools are used.

---

### **7. LLM Nodes in Cyclic Graphs**

LLM nodes often live inside **loops**:

```
Reason (LLM) → Act (Tool) → Observe → Reason
```

This enables:

* Self-correction
* Planning & replanning
* Autonomous behavior

---

### **8. Variants of LLM Nodes**

| Variant         | Purpose               |
| --------------- | --------------------- |
| Planner Node    | Generates plan        |
| Executor Node   | Performs actions      |
| Critic Node     | Evaluates output      |
| Router Node     | Selects next path     |
| Reflection Node | Improves prior result |
| Summarizer Node | Compresses context    |

---

### **9. Reliability & Production Controls**

| Feature            | Why                       |
| ------------------ | ------------------------- |
| Retries            | Handle model failures     |
| Timeouts           | Prevent hangs             |
| Parsing validation | Prevent malformed output  |
| Token limits       | Cost control              |
| Caching            | Reduce latency            |
| Model routing      | Cost-performance tradeoff |

---

### **10. Mental Model**

An LLM node is the **cognitive engine** of the graph.

> **State is the memory.
> LLM nodes are the brain.
> Edges are the nervous system.**


### Demonstration

In [1]:
from typing import TypedDict, List
import json

class State(TypedDict):
    question: str
    thoughts: List[str]
    answer: str
    confidence: float
    done: bool

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)


from langchain.tools import tool

@tool
def calculator(expr: str) -> str:
    """A simple calculator that evaluates mathematical expressions."""
    return str(eval(expr))

def reason_node(state: State):
    prompt = f"""
Question: {state['question']}
Previous thoughts: {state['thoughts']}

Think and decide next step.
If confident, answer.
If calculation required, specify calculation.
Return JSON with fields: thought, answer, confidence, done.
"""
    response = llm.invoke(prompt).content
    # Use json.loads() instead of eval() to properly parse JSON
    data = json.loads(response)

    return {
        "thoughts": state["thoughts"] + [data["thought"]],
        "answer": data["answer"],
        "confidence": data["confidence"],
        "done": data["done"]
    }

def tool_node(state: State):
    expr = state["thoughts"][-1]
    result = calculator(expr)
    return {"thoughts": state["thoughts"] + [f"Result: {result}"]}

from langgraph.graph import END

def router(state: State):
    if state["done"]:
        return END
    return "reason"

In [2]:
from langgraph.graph import StateGraph

builder = StateGraph(State)

builder.add_node("reason", reason_node)
builder.add_node("tool", tool_node)

builder.set_entry_point("reason")

builder.add_edge("tool", "reason")
builder.add_conditional_edges("reason", router, {
    "reason": "reason",
    END: END
})

graph = builder.compile()

In [3]:
result = graph.invoke({
    "question": "What is (23 * 7) + 15?",
    "thoughts": [],
    "answer": "",
    "confidence": 0.0,
    "done": False
})

print(result)


{'question': 'What is (23 * 7) + 15?', 'thoughts': ['First, I need to calculate 23 * 7, and then I will add 15 to the result.'], 'answer': 176, 'confidence': 'high', 'done': True}
